# CS-660 Semester Project Notebook 1

## Stan Rosenbaum

Fall 2017

Anaconda 5 / Python 3

Using Keras with TensorFlow as back end.


First we init stuff.

In [1]:
# Load the Basic Python Libraries
import os
import csv
import PIL
import pickle
import random
import datetime

# Load my Data Management Module
import CS660DataManagement as csDM

# load numpy
import numpy as np

# Load Keras Studd
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

from keras.callbacks import TensorBoard

from keras.utils.np_utils import to_categorical



# Other
import pydot
from IPython.display import SVG


# Not sure what this is:
# from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# Get Processed Data Directory.
processedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'PROCESSED' )
combinedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED' )
pickleDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'PICKLES' )

testImageColorFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'ICOLOR', 'TEST.png' )
testImageDepthFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'IDEPTH', 'TEST.png' )
testCSVFile = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED', 'PCLOUD', 'TEST.csv' )

combinedDataDir = os.path.join( os.getcwd(), os.pardir, 'DATA', 'COMBINED' )

imageDirs = ['ICOLOR', 'IDEPTH']
csvDirs = ['PCLOUD']

allDataFlavors = imageDirs + csvDirs

#Load main data file.
searCSVInfoFile = os.path.join( combinedDataDir, 'SEAR_DC_INFO.csv' )

csDM.CS660DataManagementCheck()

Using TensorFlow backend.


CS660DataManagementCheck Imported


## Generate the Learning and Verification sets

In [2]:
print( len( csDM.getListOfDataCSVFileKeys() ) )
csDM.reportStats()
#csDM.createLearningAndVerificationPickles()


901
Number of Everything
UPs: 278
DOWNs: 201
NAs: 204
HOLEs: 218


### Load a Training Set.
#### (Basing this all on the Coursera Course Code)



In [3]:
# def reshape(trainingSet):
#     """
#     This is to add the 'Channels' Dimension for Keras.
#     """
#     getShape = trainingSet.shape
#     newShape = getShape + (1,)
#     return trainingSet.reshape(newShape)
    

# X_train, Y_train, X_test, Y_test = csDM.load_dataset( '1', 'ICOLOR' )

# print(X_test.shape)

# print(X_test[0][0][0])

# print ("number of training examples = " + str(X_train.shape[0]))
# print ("number of test examples = " + str(X_test.shape[0]))

# X_train = reshape(X_train)
# X_test = reshape(X_test)

# print(X_test.shape)
# print(X_test[0][0][0])

# y_train_binary = to_categorical(Y_train, 4)
# y_test_binary = to_categorical(Y_test, 4)

# print ("X_train shape: " + str(X_train.shape))
# print ("Y_train shape: " + str(Y_train.shape))
# print ("y_train_binary shape: " + str(y_train_binary.shape))
# print ("X_test shape: " + str(X_test.shape))
# print ("Y_test shape: " + str(Y_test.shape))
# print ("y_test_binary shape: " + str(y_test_binary.shape))



(40, 480, 640)
0.682352941176
number of training examples = 200
number of test examples = 40
(40, 480, 640, 1)
[ 0.68235294]
X_train shape: (200, 480, 640, 1)
Y_train shape: (200, 1)
y_train_binary shape: (200, 4)
X_test shape: (40, 480, 640, 1)
Y_test shape: (40, 1)
y_test_binary shape: (40, 4)


## 2 - Building a model in Keras



In [3]:
csDM.CS660DataManagementCheck()

CS660DataManagementCheck Imported


In [4]:
import keras
from quiver_engine import server

# input image dimensions
img_rows, img_cols = 480, 640

# num_classes = 4

def buildModel( numOfNodes=48, numOfLayers=1):
    """
    Builds the basic model.
    Returns:
        A Keras NN Model

    """
    # input image dimensions
    img_rows, img_cols = 480, 640
    input_shape = (img_rows, img_cols, 1)
    num_classes = 4

    print("Building Model with ", numOfNodes, " nodes and ", numOfLayers, " layers.")

    theModel = Sequential()

    theModel.add(
        Conv2D(5,
               kernel_size=(5, 5),
               strides=3,
               activation='relu',
               input_shape=input_shape
               )
    )
    theModel.add(
        MaxPooling2D(
            pool_size=(2, 2)
        )
    )

    theModel.add(
        Conv2D(
            10,
            kernel_size=(3, 3),
            strides=2,
            activation='relu')
    )
    theModel.add(
        MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        )
    )

    theModel.add(Flatten())

    for index in range( numOfLayers ):
        theModel.add(Dense(numOfNodes))
        theModel.add(BatchNormalization())
        theModel.add(Activation('relu'))
        theModel.add(Dropout(0.25))

    theModel.add(Dense(num_classes, activation='softmax'))

    theModel.compile(
        loss=keras.losses.categorical_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=['categorical_accuracy']
    )

    theModel.summary()
    
    return theModel
    
# server.launch(model)

theModel = buildModel()


Building Model with  48  nodes and  1  layers.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 159, 212, 5)       130       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 79, 106, 5)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 39, 52, 10)        460       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 19, 26, 10)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4940)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 48)                237168    
_________________________________________________________________
batch_normalization_1 (Batch 

In [5]:
# TBoardCallback = keras.callbacks.TensorBoard(
#     log_dir='../TENSOR_LOGS/run_test', 
#     histogram_freq=0,
#     write_graph=True,
#     write_images=True
# )

# model.fit(x_train,
#           y_train_as_category,
#           batch_size=16,
#           epochs=32,
#           verbose=1,
#           validation_data=(x_test, y_test_as_category),
#           callbacks = [TBoardCallback]
#          )

tensorLogDataDir = os.path.join( os.getcwd(), os.pardir, 'TENSOR_LOGS' )

def trainModel( trainingName, theModel, x_train, y_train, x_test, y_test, num_classes=4 ):
    """
    Trains the model via given data.

    Args:
        trainingName: A name of this train [mainly to track in TensorBoard
        x_train: The X Set for Trainings
        y_train: The Y set for Testing
        x_test:  The X Set for Training/Verification
        y_test:  The Y Set for Testing/Verification

    Returns:

    """

    # Reshape the X sets.
    # Mainly for this project.because Keras/Tensor thinks in Channels.
    # And since we are using Greyscale data, we really don't have a channel.
    # So we have to 'fake' a channel
    #
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

    # Convert class vectors to binary class matrices
    y_train_as_category = to_categorical(y_train, num_classes)
    y_test_as_category = to_categorical(y_test, num_classes)

    logFilePath = os.path.join( tensorLogDataDir, trainingName )

    TBoardCallback = keras.callbacks.TensorBoard(
        log_dir=logFilePath,
        histogram_freq=0,
        write_graph=True,
        write_images=True
    )

    theModel.fit(x_train,
              y_train_as_category,
              batch_size=16,
              epochs=24,
              verbose=1,
              validation_data=(x_test, y_test_as_category),
              callbacks=[TBoardCallback]
              )
    
    return theModel

    
X_train, Y_train, X_test, Y_test = csDM.load_dataset( '0', 'IDEPTH' )

# X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
# X_test = X_test.reshape(X_train.shape[0], img_rows, img_cols, 1)

# # convert class vectors to binary class matrices
# y_train_as_category = to_categorical(y_train, num_classes)
# y_test_as_category = to_categorical(y_test, num_classes)

theModel = trainModel( "JupyterTESTRUN", theModel, X_train, Y_train, X_test, Y_test, 4)

Train on 200 samples, validate on 40 samples
Epoch 1/24
200/200 [==============================] - 3s - loss: 0.9539 - categorical_accuracy: 0.5750 - val_loss: 1.5663 - val_categorical_accuracy: 0.2500
Epoch 2/24
200/200 [==============================] - 3s - loss: 0.5863 - categorical_accuracy: 0.7800 - val_loss: 1.7589 - val_categorical_accuracy: 0.2750
Epoch 3/24
200/200 [==============================] - 3s - loss: 0.5033 - categorical_accuracy: 0.8150 - val_loss: 1.5549 - val_categorical_accuracy: 0.3750
Epoch 4/24
200/200 [==============================] - 3s - loss: 0.4425 - categorical_accuracy: 0.8800 - val_loss: 1.2527 - val_categorical_accuracy: 0.4750
Epoch 5/24
200/200 [==============================] - 3s - loss: 0.3356 - categorical_accuracy: 0.8950 - val_loss: 1.1587 - val_categorical_accuracy: 0.5000
Epoch 6/24
200/200 [==============================] - 5s - loss: 0.2832 - categorical_accuracy: 0.9400 - val_loss: 1.1105 - val_categorical_accuracy: 0.5500
Epoch 7/24
20

In [16]:
# score = model.evaluate(x_test, y_test_as_category, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# # serialize model to JSON
# model_json = model.to_json()
# with open("../MODELS/modelTEST.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("../WEIGHTS/modelTEST.h5")
# print("Saved model to disk")

def evaluateModel( theModel, x_test, y_test, num_classes):
    """
    """
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    y_test_as_category = to_categorical(y_test, num_classes)

    score = theModel.evaluate(x_test, y_test_as_category, verbose=0)
    print('General > Test loss: ', score[0], 'Test accuracy: ', score[1] )

    predictionResults = theModel.predict_classes(x_test, verbose=1)
    
    scoringList = [0, 0, 0, 0]
    scoringListAsPecents = []
    
    for index in range(len(x_test)):
        if( predictionResults[index] == y_test[index] ):
            print( index, 'Results: ', predictionResults[index], " VS ", y_test[index], "Match" )
            scoringList[ int(y_test[index]) ] = scoringList[ int(y_test[index]) ] + 1
        else:
            print( index, 'Results: ', predictionResults[index], " VS ", y_test[index], "No Match" )
    
    for element in scoringList:
        scoringListAsPecents.append( element / 10.0 )
    
    print(scoringList )
        
    return { 'SCORE': score, 'SCORELIST' : scoringListAsPecents }


X_train_Z, Y_train_Z, X_test_Z, Y_test_Z = csDM.load_dataset( '0', 'IDEPTH' )

scoringResults = evaluateModel( theModel, X_test_Z, Y_test_Z, 4)
print(scoringResults['SCORELIST'])

General > Test loss:  0.809247305989 Test accuracy:  0.675
40/40 [==============================] - 0s     
0 Results:  1  VS  [ 2.] No Match
1 Results:  1  VS  [ 2.] No Match
2 Results:  2  VS  [ 1.] No Match
3 Results:  1  VS  [ 1.] Match
4 Results:  0  VS  [ 1.] No Match
5 Results:  3  VS  [ 2.] No Match
6 Results:  2  VS  [ 2.] Match
7 Results:  3  VS  [ 1.] No Match
8 Results:  1  VS  [ 2.] No Match
9 Results:  1  VS  [ 1.] Match
10 Results:  0  VS  [ 0.] Match
11 Results:  0  VS  [ 0.] Match
12 Results:  1  VS  [ 1.] Match
13 Results:  2  VS  [ 2.] Match
14 Results:  1  VS  [ 2.] No Match
15 Results:  2  VS  [ 2.] Match
16 Results:  1  VS  [ 1.] Match
17 Results:  1  VS  [ 1.] Match
18 Results:  0  VS  [ 1.] No Match
19 Results:  0  VS  [ 2.] No Match
20 Results:  2  VS  [ 2.] Match
21 Results:  1  VS  [ 1.] Match
22 Results:  3  VS  [ 3.] Match
23 Results:  3  VS  [ 3.] Match
24 Results:  3  VS  [ 3.] Match
25 Results:  3  VS  [ 3.] Match
26 Results:  2  VS  [ 3.] No Match
27 Re

In [15]:
import pydot
import graphviz

modelsDirectory = os.path.join( os.getcwd(), os.pardir, 'MODELS' )
modelsStructsDirectory = os.path.join( os.getcwd(), os.pardir, 'MODELS_STRUCTS' )
weightsDirectory = os.path.join( os.getcwd(), os.pardir, 'WEIGHTS' )

def saveModelEverything( theModel, modelName ):
    saveModelStructure( theModel, modelName )
    saveModel( theModel, modelName )
    saveModelJSON( theModel, modelName )
    saveModelWeights( theModel, modelName )
    
    print("Model Everything Saved")
    

def saveModelStructure( theModel, modelStructureName ):
    modelStructsFilePath = os.path.join(modelsStructsDirectory, modelStructureName )
    plot_model(theModel, to_file=modelStructsFilePath)


def saveModelJSON( model, modelName ):
    """
    Saves the Model as JSON
    Args:
        model: the Keras NN Model
        modelName: the Name

    Returns:

    """
    modelFilePath = os.path.join( modelsDirectory, modelName + '.json' )
    model_json = theModel.to_json()
    with open( modelFilePath, 'w') as json_file:
        json_file.write(model_json)

        
def saveModel( model, modelName ):
    theModel.save(os.path.join( modelsDirectory, modelName ))

    
def saveModelWeights( theModel, modelName ):
    """
    Saved the Model Weights
    Args:
        weights: The Weights
        weightName: Weight Names

    Returns:

    """
    weightsFilePath = os.path.join( weightsDirectory, modelName + '.h5' )
    theModel.save_weights( weightsFilePath )
    
    
saveModelEverything( theModel, 'JUPYTER_MODEL')


Model Everything Saved


In [14]:
import pydot
import graphviz

from keras.models import load_model

modelsDirectory = os.path.join( os.getcwd(), os.pardir, 'MODELS' )
modelsStructsDirectory = os.path.join( os.getcwd(), os.pardir, 'MODELS_STRUCTS' )
weightsDirectory = os.path.join( os.getcwd(), os.pardir, 'WEIGHTS' )


def loadModel( modelName ):
    theModel = load_model( os.path.join( modelsDirectory, modelName) )
    print('MODEL Loaded.')
    return theModel


theModel = loadModel( 'JUPYTER_MODEL')

MODEL Loaded.


"NA" = 0

"UP" = 1

"DOWN" = 2

"HOLE" = 3